In [1]:
# import lib before build the nettack
import numpy as np
#//print(np.__version__)
from matplotlib import pyplot as plt
%matplotlib inline
#//import nettack

from nettack import utils
# from nettack import nettack as ntk
# from nettack import GCN

gpu_id = None # set GPU ID to use GPU computations (only for GCN/surrogate training)

In [3]:
_A_obs, _X_obs, _z_obs = utils.load_npz('data/cora.npz')

#_A_obs, _X_obs, _z_obs = utils.load_data('cora')
#_z_obs = np.loadtxt('./data-Fng/label.txt',dtype='float',delimiter=',')
#//print(_A_obs)
#//print(type(_z_obs))
#//print(_z_obs)
#//print(_z_obs.shape) # (2708, )
#//print(_X_obs)
#//print(_X_obs.shape)


_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1
lcc = utils.largest_connected_components(_A_obs)
#//print(_z_obs.shape) # (2708, )
#//print(lcc) # [0, 1, 2, 3, ..., 1998, 1999, 2000, ..., 2703, 2704, 2705, 2706, 2707]
#//print(len(lcc)) # 2485

## The Largest Connected Components makes the matrix 2708->2485
_A_obs = _A_obs[lcc][:,lcc]
#//print(_A_obs.shape) # (2485, 2485)

assert np.abs(_A_obs - _A_obs.T).sum() == 0, "Input graph is not symmetric"
assert _A_obs.max() == 1 and len(np.unique(_A_obs[_A_obs.nonzero()].A1)) == 1, "Graph must be unweighted"
assert _A_obs.sum(0).A1.min() > 0, "Graph contains singleton nodes"

_X_obs = _X_obs[lcc].astype('float32')
_z_obs = _z_obs[lcc]

#//print(_z_obs) # labels : [5 2 0 ... 2 2 2]
#//print(_z_obs.shape) # shape: (2485,)

_N = _A_obs.shape[0]



_K = _z_obs.max()+1
#//print(_z_obs.max()) # 6/6.0
#//print(_K) # 7.0
_K = int(_K)
#//print(_K) # 7
_z_obs = _z_obs.astype(dtype="int")
#//print(_z_obs) # [5 2 0 ... 2 2 2]



_Z_obs = np.eye(_K)[_z_obs]
_An = utils.preprocess_graph(_A_obs) # Normalized
sizes = [16, _K]
degrees = _A_obs.sum(0).A1



unlabeled_share = 0.8
val_share = 0.1
train_share = 1 - unlabeled_share - val_share
#//seed = 15
#//np.random.seed(seed)
split_train, split_val, split_unlabeled \
    = utils.train_val_test_split_tabular(np.arange(_N),
                                         train_size=train_share,
                                         val_size=val_share,
                                         test_size=unlabeled_share,
                                         stratify=_z_obs
                                        )

Selecting 1 largest connected components
